In [4]:
import pandas as pd

#데이터셋 로드
df = pd.read_csv('LOL_match data_Combined.csv')

#티어-matchID당 데이터가 10개가 아닌 경우는 제외(챔피언 정보가 다 들어가야 10개이므로)
#그냥 matchID로 보면 이상치 존재(티어는 다른데 매치ID가 같은 경우 → 데이터 수집 과정에서 발생)
df_grouped = df.groupby(['Tier','matchId']).count()
invalid_matchIds = df_grouped[df_grouped['gameType'] != 10].index.tolist()

#티어-matchID 조합에서는 이상치가 없는 것 확인
len(invalid_matchIds)

#팀 포지션이 비어있는 경우의 매치ID를 제외
matchid_invalid_position = df[df['teamPosition'].isnull()]['matchId'].unique()
df = df[~df['matchId'].isin(matchid_invalid_position)]

#경기시간 600초 초과 & 경험치 0 초과인 마스크 적용
mask = (df['timePlayed'] > 600) & (df['champExperience'] > 0)
df_filtered = df[mask]
df_filtered['season'] = df_filtered['gameType'].str[:2]
season_mask = (df_filtered['season'] == '14')
df_filtered = df_filtered[season_mask]

# matchID별 win 컬럼이 TRUE와 FALSE의 비율이 5:5가 되는 matchId만 추출
balanced_df = df_filtered.groupby('matchId').filter(lambda x: (x['win'].sum() == 5) & (x['win'].count() - x['win'].sum() == 5))

C:\Users\alaru\AppData\Local\Temp\ipykernel_27200\3381541291.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['season'] = df_filtered['gameType'].str[:2]


In [5]:
# teamPosition이 JUNGLE 또는 TOP인 데이터만 추출하고 기본적인 컬럼만 포함
df_jungle_top = balanced_df[balanced_df['teamPosition'].isin(['JUNGLE', 'TOP'])][['matchId', 'teamPosition', 'championName', 'win', 'teamId']]
df_jungle_top

,matchId,teamPosition,championName,win,teamId
80,KR_7115344179,TOP,Kennen,False,100
84,KR_7115344179,JUNGLE,Skarner,False,100
85,KR_7115344179,TOP,Gragas,True,200
89,KR_7115344179,JUNGLE,Nocturne,True,200
91,KR_7160471688,TOP,DrMundo,True,100
...,...,...,...,...,...
4663119,KR_7152013931,TOP,DrMundo,True,200
4663120,KR_7062814093,TOP,Kalista,False,100
4663121,KR_7062814093,JUNGLE,Karthus,False,100
4663125,KR_7062814093,TOP,Heimerdinger,True,200


In [6]:
# df_jungle_mid의 각 matchId에 대해 조건을 확인하는 함수
def validate_match_groups(df):
    valid_match_ids = []
    for match_id, group in df.groupby('matchId'):
        if len(group) == 4:
            team_ids = group['teamId'].values
            wins = group['win'].values
            if (list(team_ids).count(100) == 2 and list(team_ids).count(200) == 2 and
                ((wins[:2] == [True, True]).all() and (wins[2:] == [False, False]).all()) or
                ((wins[:2] == [False, False]).all() and (wins[2:] == [True, True]).all())):
                valid_match_ids.append(match_id)
    return valid_match_ids

# 유효한 matchId 리스트를 확인
valid_match_ids = validate_match_groups(df_jungle_top)

# 유효한 matchId를 가진 데이터프레임 필터링
valid_df = df_jungle_top[df_jungle_top['matchId'].isin(valid_match_ids)]
valid_df

,matchId,teamPosition,championName,win,teamId
80,KR_7115344179,TOP,Kennen,False,100
84,KR_7115344179,JUNGLE,Skarner,False,100
85,KR_7115344179,TOP,Gragas,True,200
89,KR_7115344179,JUNGLE,Nocturne,True,200
91,KR_7160471688,TOP,DrMundo,True,100
...,...,...,...,...,...
4663119,KR_7152013931,TOP,DrMundo,True,200
4663120,KR_7062814093,TOP,Kalista,False,100
4663121,KR_7062814093,JUNGLE,Karthus,False,100
4663125,KR_7062814093,TOP,Heimerdinger,True,200


In [8]:
# 같은 matchId에서 탑과 정글의 이름조합 추출
combinations = []

for match_id, group in valid_df.groupby('matchId'):
    for team_id in [100, 200]:
        team_group = group[group['teamId'] == team_id]
        junglers = team_group[team_group['teamPosition'] == 'JUNGLE']['championName'].values
        tops = team_group[team_group['teamPosition'] == 'TOP']['championName'].values
        win = team_group['win'].values[0]  # 해당 팀의 승리 여부
        if len(junglers) > 0 and len(tops) > 0:
            combinations.extend([(jungle, top, win) for jungle in junglers for top in tops])

# DataFrame으로 변환
combinations_df = pd.DataFrame(combinations, columns=['JungleChampion', 'TopChampion', 'win'])

# 조합 별 승률 계산
win_rates = combinations_df.groupby(['JungleChampion', 'TopChampion']).agg(
    games_played=('win', 'count'),
    wins=('win', 'sum')
).reset_index()

win_rates['win_rate'] = win_rates['wins'] / win_rates['games_played']
win_rates

,JungleChampion,TopChampion,games_played,wins,win_rate
0,Aatrox,Ahri,1,0,0.000000
1,Aatrox,Akali,4,1,0.250000
2,Aatrox,Aurora,4,2,0.500000
3,Aatrox,Azir,1,0,0.000000
4,Aatrox,Belveth,1,0,0.000000
...,...,...,...,...,...
11859,Zyra,Zac,91,55,0.604396
11860,Zyra,Zed,31,13,0.419355
11861,Zyra,Zeri,18,10,0.555556
11862,Zyra,Ziggs,1,0,0.000000
